In [2]:
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint
from skimage import measure

from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
from datetime import datetime as dt
import pickle

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings

## current script object
thismodule = sys.modules[__name__]

# Own settings
from settings import *

workdir = os.getcwd()
moduledir, fcndir = defineDir(workdir)

Own modules available: ['casestudy', 'conditionalstats', 'conditionalstats_chunked', 'lifecycle_mapping', 'plot1D', 'plot2D', 'plotCondPDFs']
Own modules available: ['PrecipGrid', 'fcns_load_DYAMOND_SAM', 'fcns_load_RCEMIP_SAM', 'fcns_movie', 'fcns_multiscale_extremes', 'load_TOOCAN_DYAMOND', 'load_TOOCAN_DYAMOND_modif', 'load_TOOCAN_DYAMOND_modif_BF', 'fcns_load_DYAMOND_SAM_Tb_Feng']


In [4]:
%autoreload 2

# to access segmentation files and simulation outputs
from fcns_load_DYAMOND_SAM import *
from fcns_load_DYAMOND_SAM_Tb_Feng import *
# display functions
from fcns_movie import *
# plotting
from plot1D import *

In [5]:
df = pd.read_csv(os.path.join(DIR_DATA,'relation_2_table_UTC_dyamond_segmentation.csv'))
df.sort_values(by='UTC',ignore_index=True,inplace=True)

In [18]:
df

,Unnamed: 0,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,img_seg_path,str_code,Unnamed: 0.1,Unnamed: 0.1.1,year,month,day,hour,minute,path_dyamond,UTC
0,476,476.0,476.0,476.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-1-30,2.0,2.0,2016.0,8.0,1.0,1.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000000720,17014.03
1,443,443.0,443.0,443.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-2-0,3.0,3.0,2016.0,8.0,1.0,2.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000000960,17014.04
2,468,468.0,468.0,468.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-2-30,4.0,4.0,2016.0,8.0,1.0,2.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000001200,17014.05
3,432,432.0,432.0,432.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-3-0,5.0,5.0,2016.0,8.0,1.0,3.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000001440,17014.06
4,437,437.0,437.0,437.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-8-1-3-30,6.0,6.0,2016.0,8.0,1.0,3.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000001680,17014.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,1295,1295.0,1295.0,1295.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-9-22-30,1916.0,1916.0,2016.0,9.0,9.0,22.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460080,17053.45
1915,1315,1315.0,1315.0,1315.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-9-23-0,1917.0,1917.0,2016.0,9.0,9.0,23.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460320,17053.46
1916,1313,1313.0,1313.0,1313.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-9-23-30,1918.0,1918.0,2016.0,9.0,9.0,23.0,30.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460560,17053.47
1917,1304,1304.0,1304.0,1304.0,data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOB...,2016-9-10-0-0,1919.0,1919.0,2016.0,9.0,10.0,0.0,0.0,DYAMOND_9216x4608x74_7.5s_4km_4608_0000460800,17053.48


## Check relation table has correct matching

It appears

In [61]:
# exmaple index
i_ex = 93

In [62]:
print(df.iloc[i_ex].str_code), df.iloc[i_ex].img_seg_path, df.iloc[i_ex].path_dyamond

2016-8-3-0-0


(None,
 'data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/2016_08_02/ImageSegTOOCAN_20160802-48.nc',
 'DYAMOND_9216x4608x74_7.5s_4km_4608_0000023040')

In [63]:
path_data = '/bdd/DYAMOND/SAM-4km/OUT_2D/%s.Precac.2D.nc'%df.iloc[i_ex].path_dyamond
timestamp = int(df.iloc[i_ex].path_dyamond.split('_')[-1])
delta_t = timestamp*7.5/3600
print(path_data)
print('timestamp:',timestamp)
print('delta_t =',delta_t)

/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000023040.Precac.2D.nc
timestamp: 23040
delta_t = 48.0


In [64]:
datetime_ref = dt.datetime(year=2016,month=8,day=1)
print(datetime_ref)
delta_time = dt.timedelta(hours=delta_t)
print(delta_time)
datetime_ex = datetime_ref + delta_time
print(datetime_ex)

2016-08-01 00:00:00
2 days, 0:00:00
2016-08-03 00:00:00


In [65]:
! ncdump -v time /bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000024720.Precac.2D.nc

netcdf DYAMOND_9216x4608x74_7.5s_4km_4608_0000024720.Precac.2D {
dimensions:
	time = UNLIMITED ; // (1 currently)
	lat = 4608 ;
	lon = 9216 ;
variables:
	float Precac(time, lat, lon) ;
		Precac:long_name = "Surface Accum Precip." ;
		Precac:units = "mm" ;
	float lat(lat) ;
		lat:units = "degrees_north" ;
	float lon(lon) ;
		lon:units = "degrees_east" ;
	float time(time) ;
		time:units = "day" ;
		time:long_name = "time" ;

// global attributes:
		:history = "Sat Jul 28 09:49:00 2018: ncks -v Precac /gpfs/p/unsb0007/SAM7.1_DYAMOND/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000024720.2Dcom_001_0.nc /glade/scratch_new/marat/SAM7.1_DYAMOND/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000024720.Precac.2D.nc" ;
		:NCO = "netCDF Operators version 4.7.4 (http://nco.sf.net)" ;
data:

 time = 216.1458 ;
}


In [70]:
! ncdump -v time /data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/2016_08_02/ImageSegTOOCAN_20160802-48.nc

netcdf ImageSegTOOCAN_20160802-48 {
dimensions:
	latitude = 2072 ;
	longitude = 9216 ;
	time = UNLIMITED ; // (1 currently)
variables:
	float MCS_label(time, latitude, longitude) ;
		MCS_label:units = " " ;
		MCS_label:standard_name = "MCS_label" ;
		MCS_label:long_name = "MCS labellisation" ;
		MCS_label:_FillValue = -998.f ;
		MCS_label:missing_value = -999.f ;
	float latitude(latitude) ;
		latitude:units = "degrees_north" ;
		latitude:long_name = "latitude" ;
	float longitude(longitude) ;
		longitude:units = "degrees_east" ;
		longitude:long_name = "longitude" ;
		longitude:FlagDegEst = 0 ;
	double time(time) ;
		time:standard_name = "time" ;
		time:long_name = "slot begin time" ;
		time:units = "seconds since 1970-01-01 00:00:00 UTC" ;

// global attributes:
		:title = "TOOCAN segmented images - Gridded data 04 km" ;
		:image_time = "2016-08-02-T23-30-00 UTC" ;
		:slot_start_time = "23-30-00 UTC" ;
		:slot_end_time = "00-00-00 UTC" ;
		:platform = "" ;
		:channel = "Infrared " ;
		

In [71]:
datetime_base_ref = dt.datetime(year=1970,month=1,day=1)
print(datetime_base_ref)
# delta_time_seg = dt.timedelta(seconds = 1470193200)
delta_time_seg = dt.timedelta(seconds = 1470180600)
print(delta_time_seg)
datetime_ex_seg = datetime_base_ref + delta_time_seg 
print(datetime_ex_seg)

1970-01-01 00:00:00
17015 days, 23:30:00
2016-08-02 23:30:00


# Goal: Compare old and new mask with a changed Tb threshold to detect MCSs

### Make new relation table (check that the relation table is correct)

In [16]:
df.iloc[46]['path_dyamond']

'DYAMOND_9216x4608x74_7.5s_4km_4608_0000012000'

In [17]:
i_t_min = 0
i_t_max = 1919

In [41]:
! ls /bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_*.Precac.2D.nc | head -2

/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000000240.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000000480.Precac.2D.nc
ls: write error: Broken pipe


In [37]:
! ncdump -v time /bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000000720.Precac.2D.nc | tail -2 | head -1

 time = 214.0625 ;


In [11]:
from datetime import timezone

In [11]:
def index2date(i_t):
    
    # reference
    dict_date_ref = {'year' : 2016, 'month' : 8, 'day' : 1} # ! edit BF
    datetime_ref = dt.datetime(dict_date_ref['year'], dict_date_ref['month'], dict_date_ref['day'])
    timestamp_ref = datetime_ref.timestamp()
    
    # find date
    i_t_in_seconds = (i_t+1) * 30 * 60 # ! edit BF: advance time step
    timezone_weird_lag_to_watch = 2*60*60 #2hours
    timestamp = timestamp_ref + i_t_in_seconds + timezone_weird_lag_to_watch
    date = dt.datetime.utcfromtimestamp(timestamp)
    
    return date

def get_filename_tb_feng(i_t):
    
    root = "/bdd/MT_WORKSPACE/lgouttes/MODELS/DYAMOND/Summer/SAM/TOOCAN_Olr-Tb_Feng/TOOCAN_v2.07/GLOBAL/2016/"
    
    date = index2date(i_t)
    
    # strings
    date_time_str = date.strftime("%Y-%m-%d-%H-%M")
    date_str = date.strftime("%Y_%m_%d")
    date_str_no_underscore = date.strftime("%Y%m%d")
    
    hours = int(date.strftime("%H"))
    minutes = int(date.strftime("%M"))
    n_half_hours = int(2*hours + minutes/30 + 1)
    dir_path = os.path.join(root, date_str)
    file_root= "mcs_mask_TOOCAN_SAM_"+date_str_no_underscore+'-'+str(n_half_hours).zfill(3)+'.nc'
    filename = os.path.join(dir_path, file_root)
    
    return filename

def timeStep2Date(dt_str):
    
    # reference
    dict_date_ref = {'year' : 2016, 'month' : 8, 'day' : 1} # ! edit BF
    datetime_ref = dt.datetime(dict_date_ref['year'], dict_date_ref['month'], dict_date_ref['day'])
    
    timestamp_ref = datetime_ref.timestamp()
    
    # find date
    time_step = int(int(dt_str)*7.5)
    timezone_weird_lag_to_watch = 2*60*60 #2hours
    timestamp = timestamp_ref + time_step + timezone_weird_lag_to_watch
    date = dt.datetime.utcfromtimestamp(timestamp)
    
    return date

In [ ]:
for i_t in range(0,48):

    print(i_t,df.iloc[i_t]['img_seg_path'],df.iloc[i_t]['path_dyamond'])

In [ ]:
for i_t in range(0,1920):

    filepath = get_filename_tb_feng(i_t)
    if len(glob.glob(filepath)) > 0:
        print(i_t,filepath)

In [13]:
os.path.basename(os.path.dirname(get_filename_tb_feng(10)))

'2016_08_01'

In [14]:
i_t = 0
dt_str = '0000000240'

print(index2date(i_t))
date = timeStep2Date(dt_str) 
print(timeStep2Date(dt_str))


2016-08-01 00:30:00
2016-08-01 00:30:00


# Create timetable

In [154]:
index = list(range(0,1920))
img_seg_path = [get_filename_tb_feng(i_t) for i_t in index]
str_code = [index2date(i_t).strftime("%Y-%m-%d-%H-%M") for i_t in index]
time_stamp = [None for i_t in index]
path_dyamond = [None for i_t in index]

In [155]:
file_root = 'DYAMOND_9216x4608x74_7.5s_4km_4608_%s.%s.2D.nc'
varid = 'Precac'
stored_files = glob.glob(os.path.join(DIR_DYAMOND,file_root%('*',varid)))
stored_files.sort()

for i in index:
    
    s_c = str_code[i]
    delta_t = dt.datetime.strptime(s_c,'%Y-%m-%d-%H-%M')-dt.datetime(year=2016,month=8,day=1)
    time_stamp[i] = str(int((delta_t.days*86400 + delta_t.seconds)/7.5)).rjust(10,'0')
    file_name = os.path.join(DIR_DYAMOND,file_root%(time_stamp[i],varid))
    if file_name in stored_files:
        path_dyamond[i] = file_name

In [157]:
d = pd.DataFrame({'img_seg_path': pd.Series(img_seg_path,
                                  index=index),
                 'str_code': pd.Series(str_code,
                                  index=index),
                 'time_stamp': pd.Series(time_stamp,
                                  index=index),
                 'path_dyamond': pd.Series(path_dyamond,
                                  index=index)})

In [161]:
# Check

In [158]:
! ls /bdd/DYAMOND/SAM-4km/OUT_2D/*Precac* | head -60 | tail -20

/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000009840.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000010080.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000010320.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000010560.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000010800.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000011040.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000011280.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000011520.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000011760.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000012000.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000012240.Precac.2D.nc
/bdd/DYAMOND/SAM-4km/OUT_2D/DYAMOND_9216x46

In [12]:
d[1900:]

NameError: name 'd' is not defined